In [1]:
import pandas as pd

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [9]:
# train = spark.read.csv('mitbih_train.csv')
#train.limit(10).toPandas()
train = pd.read_csv('mitbih_train.csv', header=None)
train.head()

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.977941,0.926471,0.681373,0.245098,0.154412,0.191176,0.151961,0.085784,0.058824,0.049020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
cols = train.columns.tolist()
cols = cols[-1:] + cols[:-1]
train = train[cols]


In [17]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(train, test_size=0.2, random_state=42)

In [19]:
train_df.shape
valid_df.shape

(17511, 188)

In [48]:
train_df[187] = train_df[187].astype(int)
train_df.head(5)

,187,0,1,2,3,4,5,6,7,8,...,177,178,179,180,181,182,183,184,185,186
77102,2,0.996226,1.000000,0.742138,0.355975,0.250314,0.252830,0.226415,0.191195,0.173585,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65922,0,1.000000,0.623301,0.238835,0.205825,0.097087,0.000000,0.029126,0.118447,0.209709,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11365,0,0.987805,0.939024,0.550813,0.069106,0.018293,0.284553,0.434959,0.424797,0.406504,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36722,0,0.881068,0.638350,0.055825,0.000000,0.101942,0.177184,0.218447,0.245146,0.257282,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23017,0,1.000000,0.823204,0.593923,0.367403,0.220994,0.116022,0.049724,0.046961,0.024862,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_df[187].unique()

In [63]:
train_df.to_csv('train.csv', header=False, index=False)
valid_df.to_csv('validation.csv', header=False, index=False)

In [23]:
import sagemaker
import boto3
from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput

region = boto3.Session().region_name

sess = sagemaker.Session()

prefix = 'xgboost_classification'
bucket = sess.default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [65]:
!aws s3 cp train.csv s3://$bucket/$prefix/train/
!aws s3 cp validation.csv s3://$bucket/$prefix/validation/

upload: ./train.csv to s3://sagemaker-us-east-2-290425241802/xgboost_classification/train/train.csv


In [67]:
!aws s3 ls s3://$bucket/$prefix/validation/

2023-11-30 17:44:28   42742588 valid.csv


In [27]:
from sagemaker.image_uris import retrieve

container = retrieve(framework="xgboost", region=region, version="1.7-1")

In [68]:
hyperparameters = {
    "objective": "multi:softmax",
    "num_class": "5",
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.7",
    "num_round":"50"}

output_path="s3://{}/{}/output/".format(bucket,prefix)
xgb_model=  sagemaker.estimator.Estimator(image_uri=container, 
                                hyperparameters=hyperparameters,
                                role=sagemaker.get_execution_role(),
                                instance_count=1, 
                                instance_type='ml.m4.xlarge', 
                                output_path=output_path)

train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/train.csv".format(bucket,prefix), content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validation/validation.csv".format(bucket,prefix), content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

xgb_model.fit(inputs=data_channels, logs=False)

print('ready for hosting!')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-11-30-17-44-35-330



2023-11-30 17:44:35 Starting - Starting the training job....
2023-11-30 17:45:01 Starting - Preparing the instances for training...............
2023-11-30 17:46:22 Downloading - Downloading input data......
2023-11-30 17:46:57 Training - Downloading the training image...........
2023-11-30 17:47:58 Training - Training image download completed. Training in progress..................................
2023-11-30 17:50:50 Uploading - Uploading generated training model..
2023-11-30 17:51:07 Completed - Training job completed
ready for hosting!


In [69]:
predictor = xgb_model.deploy(initial_instance_count=1,
                serializer = sagemaker.serializers.CSVSerializer(),
                instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-11-30-17-51-09-292
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-11-30-17-51-09-292
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-11-30-17-51-09-292


------!